## Quaternion PyTorch - Basic mechanisms

In [12]:
import torch
from htorch import quaternion

### 1 - Quaternion tensors

A quaternion number is represented by:

$$
x = a + bi + cj + dk
$$

where $a$, $b$, $c$, and $d$ are real values, and $i$, $j$, $k$ are the imaginary parts. A `QuaternionTensor` extends the standard PyTorch `tensor` to handle quaternion values, by specifying the real and imaginary components during initialization:

In [ ]:
# Simple scalar quaternion
x = quaternion.QuaternionTensor([0.0, 0.3, 0.4, 0.5])
x

In [ ]:
# Mini-batch of two scalar quaternions
x = quaternion.QuaternionTensor(torch.rand(2, 4))
print(x)

In [ ]:
# A vector with 4 quaternions
y = quaternion.QuaternionTensor(torch.rand(16))
print(y)

All standard quaternion operations can be applied on the tensor (see `QuaternionTensor` for a full list):

In [ ]:
# Conjugation
print(x.conj)

In [ ]:
# Element-wise norm
print(x.norm)

In [ ]:
# Element-wise angle
print(x.theta)

In [ ]:
# Quaternion multiplication (Hamilton product)
print(x * x)

In [ ]:
# Quaternion matrix multiplication
print(x.t() @ x)

Importantly, quaternion tensors and real-valued tensors are interoperable (real-valued tensors being casted to quaternion tensors with 0 imaginary parts):

In [ ]:
# Quaternion scalar multiplication
print(x * torch.rand(2))

### 2 - Quaternion gradients

Gradients can be computed with the PyTorch autograd mechanisms:

In [ ]:
x = quaternion.QuaternionTensor(torch.rand(2, 4))
x.requires_grad = True
y = x.norm().sum()
y.backward()

In [ ]:
a = torch.rand((2,4))
a.requires_grad=True
b = a.sum()
b.backward()

In [ ]:
print(x.grad)

### 3 - Quaternion-valued layers

We also provide a number of quaternion-valued layers to implement quaternion neural networks:

In [ ]:
from torch import nn
from qtorch.layers import QLinear

In [ ]:
model = nn.Sequential(
    QLinear(4, 20),
    nn.ReLU(),
    QLinear(20, 1)
)

In [ ]:
x = quaternion.QuaternionTensor(torch.rand(2, 16))
print(model(x))

In [ ]:
model(x).shape

We also provide layers to easily integrate quaternion-valued and real-valued blocks:

In [ ]:
from qtorch.layers import QuaternionToReal
from torch.nn import Softmax

In [ ]:
model = nn.Sequential(
    QLinear(4, 10),
    QuaternionToReal(10), # Take the absolute value
    Softmax()
)

In [ ]:
model(x).shape